In [1]:
import pandas as pd

In [26]:
txt = 'compare.txt'

In [7]:
df = pd.read_fwf(txt)

In [27]:
df

,00002649_00000001_slap_S_01.png,00002649_00000001_roll_U_01.png
0,00002649_00000001_roll_V_01.png,00002312_00000001_roll_U_01.png
1,00002649_00000001_roll_U_01.png,00002649_00000001_roll_V_01.png
2,00002649_00000001_roll_V_01.png,00002649_00000001_slap_S_01.png
3,00002312_00000001_roll_U_01.png,00002649_00000001_slap_S_01.png


In [50]:
df = pd.read_csv(txt, sep=' ')

In [51]:
df = df.reset_index

In [52]:
df

<bound method DataFrame.reset_index of                                                                5
00002649_00000001_slap_S_01.png  00002649_00000001_roll_U_01.png
00002649_00000001_roll_V_01.png  00002312_00000001_roll_U_01.png
00002649_00000001_roll_U_01.png  00002649_00000001_roll_V_01.png
00002649_00000001_roll_V_01.png  00002649_00000001_slap_S_01.png
00002312_00000001_roll_U_01.png  00002649_00000001_slap_S_01.png>

In [57]:
def read_txt_dataframe(txtPath, sep=' '):
    df = pd.read_csv(txtPath, sep=sep)
    df = df.reset_index()
    df.columns = ['image1', 'image2']
    return df  

In [58]:
df = read_txt_dataframe(txt)

In [60]:
from sklearn.model_selection import KFold


In [61]:
kf = KFold(n_splits=5)
split = kf.split(df)

In [2]:
import timm
import torch
import torch.nn as nn

/home/nakano/.virtualenvs/brickbanker/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/nakano/.virtualenvs/brickbanker/lib/python3.8/site-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)


In [79]:
encoder = timm.create_model(model_name='tf_efficientnetv2_b0', 
                                    pretrained=True,
                                    features_only=True)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-effv2-weights/tf_efficientnetv2_b0-c7cc451f.pth" to /home/nakano/.cache/torch/hub/checkpoints/tf_efficientnetv2_b0-c7cc451f.pth


In [80]:
out_feature = encoder(torch.randn(1, 3, 224, 224))


In [81]:
n_out = 1
for dim in out_feature[-1].shape:
    n_out *= dim

In [82]:
n_out

9408

In [83]:
def dimensionality_reductor():
    return nn.Sequential(
        nn.Flatten(),
        nn.Linear(n_out, 512), nn.ReLU(inplace = True),
        nn.Linear(512, 256), nn.ReLU(inplace=True),
        nn.Linear(256, 64)
    )
    
def forward(input1, input2):
    output1 = encoder(input1)[-1]
    output1 = dimensionality_reductor()(output1)
    output2 = encoder(input2)[-1]
    output2 = dimensionality_reductor()(output2)

    return output1, output2

In [84]:
i1, i2 = torch.randn(3, 3, 224, 224), torch.randn(3, 3, 224, 224)

In [85]:
o1, o2 = forward(i1, i2)

In [86]:
o1.shape

torch.Size([3, 64])

In [26]:
encoder.reset_classifier(0)

AttributeError: 'FeatureListNet' object has no attribute 'reset_classifier'

In [21]:
#encoder

In [34]:
encoder.feature_info.channels()

[64, 128, 256, 512, 512, 512]

In [35]:
#encoder.eval()
encoder(torch.randn(1, 3, 224, 224))[-1].shape

torch.Size([1, 512, 7, 7])

In [53]:
512*7*7

25088

In [98]:
imPath = '../registered_images_v1'
imList = os.listdir(imPath)

In [106]:
import cv2
import numpy as np
import os
im = cv2.imread(os.path.join(imPath, imList[0]))
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB).astype(np.float32)

In [107]:
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90, Perspective,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue, Affine,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, PiecewiseAffine, RandomResizedCrop,
    Sharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout,
    ShiftScaleRotate, CenterCrop, Resize, ElasticTransform, ImageCompression,
)
from albumentations.pytorch import ToTensorV2

In [164]:
def get_train_transforms():
    return Compose([
        Resize(224, 224, interpolation=cv2.INTER_CUBIC, p=1.),
        # Transpose(p=0.5),
        HorizontalFlip(p=0.5),
        VerticalFlip(p=0.5),
        # ShiftScaleRotate(p=0.5),
        # Perspective(p=0.5),
        #ElasticTransform(p=0.5),
        #GridDistortion(p=0.5),
        #CLAHE(p=0.5),
        #Cutout(p=0.25),
        #MotionBlur(p=0.25),
        #ImageCompression(p=0.5, quality_lower=50, quality_upper=100),
        # Affine(scale=[0.5, 1.5], p=0.5),
        Sharpen(p=0.25),
        #HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
        #RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5),
        #CoarseDropout(p=0.5),
        ToTensorV2(p=1.0),
    ], p=1.)

In [165]:
_im = get_train_transforms()(image=im)['image']

In [166]:
_im = _im.unsqueeze(0)
_im.shape

torch.Size([1, 3, 224, 224])

In [167]:
_im.shape

torch.Size([1, 3, 224, 224])

In [168]:
encoder(_im)[-1].shape

torch.Size([1, 512, 7, 7])

In [88]:
for param in encoder.parameters():
            param.requires_grad = False

In [150]:
encoder.eval()
encoder(torch.randn(1, 3, 224, 224))[-1].shape

torch.Size([1, 512, 7, 7])

In [160]:
encoder.feature_info[-1]

{'num_chs': 512, 'reduction': 32, 'module': 'features.36'}

In [139]:
encoder.feature_info.channels()

[64, 128, 256, 512, 512, 512]

In [172]:
class SiameseNetworkTimmBackbone(nn.Module):
    def __init__(self, model_name:str):
        super().__init__()
        self.encoder = timm.create_model(model_name=model_name, 
                                       pretrained=True,
                                       features_only=True)
        for param in self.encoder.parameters():
            param.requires_grad = False
        n_out = self.encoder.feature_info[-1]['num_chs']

        self.dimensionality_reductor = nn.Sequential(
            nn.Flatten(),
            nn.Linear(25088, 512), nn.ReLU(inplace = True),
            nn.Linear(512, 256), nn.ReLU(inplace=True),
            nn.Linear(256, 64)
        )

    def forward(self, input1, input2):
        print(input1.shape)
        print(input1.dtype)
        
        output1 = self.encoder(input1)[-1]
        print(output1.shape)
        output1 = self.dimensionality_reductor(output1)
        output2 = self.encoder(input2)[-1]
        output2 = self.dimensionality_reductor(output2)

        return output1, output2

In [173]:
tm = SiameseNetworkTimmBackbone('vgg19')

In [174]:
tm(_im, _im)

torch.Size([1, 3, 224, 224])
torch.float32
torch.Size([1, 512, 7, 7])


(tensor([[ 1.3377,  0.7286,  1.5252,  2.7763, -5.9279,  2.1389, -1.2336,  1.2438,
           0.3906,  3.6307, -1.7119,  0.0957, -0.5849, -0.9776,  0.9360, -2.1718,
           1.4305, -3.3072,  2.9513, -2.1385, -2.0622, -0.8845, -0.9819, -0.0067,
          -0.9565,  1.3756,  2.0640, -1.2633, -1.0078, -0.3105, -2.1146, -0.6408,
          -1.8333,  4.9663,  1.3228,  1.0289,  2.6461,  0.1993,  2.0960, -2.8636,
          -1.0461,  0.8179, -0.4607,  2.4057, -2.1714, -0.0397, -0.3311, -0.8036,
           0.5419, -0.1144,  1.0472,  0.5534, -0.3284, -1.2199,  0.8545,  1.7818,
          -1.2975,  1.1899,  0.8478, -0.8254, -0.4918,  0.7887,  4.6003,  2.7350]],
        grad_fn=<AddmmBackward0>),
 tensor([[ 1.3377,  0.7286,  1.5252,  2.7763, -5.9279,  2.1389, -1.2336,  1.2438,
           0.3906,  3.6307, -1.7119,  0.0957, -0.5849, -0.9776,  0.9360, -2.1718,
           1.4305, -3.3072,  2.9513, -2.1385, -2.0622, -0.8845, -0.9819, -0.0067,
          -0.9565,  1.3756,  2.0640, -1.2633, -1.0078, -0.310

In [175]:
import torch.functional as F

In [ ]:
torch.nn.functional.pa

In [87]:
import yaml

In [88]:
def read_config_yaml(config_path):
    with open(config_path, "r") as yaml_file:
        config = yaml.load(yaml_file, Loader=yaml.FullLoader)
    return config

In [89]:
config = read_config_yaml('config.yaml')

In [95]:

def config_flatten(config, fconfig):
    for key in config:
        if isinstance(config[key], dict):
            fconfig = config_flatten(config[key], fconfig)
        else:
            fconfig[key] = config[key]
    return fconfig

In [96]:
fconfig = {}
fconfig = config_flatten(config, fconfig)

In [97]:
fconfig

{'data_path': 'registred_images_v2',
 'data_nlets': 'compare_fp_v1.txt',
 'network': 'tf_efficientnetv2_b0',
 'train_batch': 128,
 'valid_batch': 32,
 'num_workers': 1,
 'image_size': 224,
 'nchannels': 3,
 'num_splits': 5,
 'epochs': 10,
 'lr': 0.001,
 'scheduler': 'stepLr',
 'accum_iter': 1,
 'verbose_step': 1,
 'margin': 2.0,
 'contrastive_thresh': 1.1}

In [2]:
style = 'ADD'
toadd = 10
y = [style for i in range(toadd)]

In [3]:
y

['ADD', 'ADD', 'ADD', 'ADD', 'ADD', 'ADD', 'ADD', 'ADD', 'ADD', 'ADD']